# D&D Dice Fraud Detection Software: Neural Network Training

## 0. Import

NOTE: Make sure to switch Python / Anaconda environments to the TensorFlow one.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

## 1. Load Dataset and Train/Validate/Test Split

In [ ]:
fulldataset=pd.read_pickle

## 2. Define Neural Network Metaparameters

Do not, I repeat, do **NOT** mess with the Metaparameters and run the training on the same data set. Create an entirely new batch of data points with the data generation script, to avoir all possibility of overfitting.

## 3. Train Network

## 4. Test Model

## 5. Export Model